**GRUPO 2**

> **Integrantes**:
>
> - Roberto Perez Esteban
> - Sofia Gomez Ramirez
> - Agustin Arganin Castillo

Proyecto de análisis exploratorio de datos de inmuebles

In [3]:
import csv
import fnmatch
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy
import seaborn as sns

# Proyecto EDA
## Análisis de Viviendas

In [4]:
print(os.getcwd())

c:\Users\asgr2\Desktop\Bootcamp_DS\EDA_Inmuebles_Idealista\src\notebooks


Función para rastrear directorios y archivos en ellos.

In [5]:
def check_extension(path):
	if fnmatch.fnmatch(path, "*.csv"):
		return ("csv", pd.read_csv(path))
	if fnmatch.fnmatch(path, "*.json"):
		return ("json", pd.read_json(path))
	if fnmatch.fnmatch(path, "*.txt"):
		return "txt"
	if fnmatch.fnmatch(path, "*.xlsx"):
		return ("xlsx", pd.read_excel(path))


def load_file(path, data):
	if not os.path.isfile(path):
		raise ValueError("La ruta es un archivo.")

	extension, dataset = check_extension(path)
	data[extension] = dataset

	return extension

In [6]:
path = os.getcwd()
def load_datasets(path, data):
	print("Current Working Directory:\n\t-", path)
	if path is None or path == "":
		raise ValueError("Ruta inválida o inexistente.")

	if os.path.isfile(path):
		load_file(path, data)
	elif not os.path.isdir(path):
		raise ValueError(f"El path no corresponde a un directorio o archivo. Path: {path}")

	with os.scandir(path) as directories:
		for directory in directories:
			try:
				dir = f"{path}/{directory.name}"
				os.chdir(dir)
				load_datasets(f"{path}/{directory.name}", data)
			except Exception as e:
				raise ValueError("Error:", e)

In [7]:
os.getcwd()

'c:\\Users\\asgr2\\Desktop\\Bootcamp_DS\\EDA_Inmuebles_Idealista\\src\\notebooks'

In [8]:
datasets = []
base_path = "../data"

### Cargar datos de múltiples datasets
#### 21-04-2023

Dentro de la carpeta encontramos archivos `.txt` y `.csv`.

In [9]:
path = f"{base_path}/23-04-21"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/23-04-21
['23-04-21 Descarga Idealista.txt', '23-04-21Datos2000(4).csv']


##### Archivo: **23-04-21 Descarga Idealista.txt**

Cargamos el dataset.

In [10]:
dir = f"{path}/{files[0]}"
print("Directory:", dir)
with open(dir, "r", encoding="utf-8") as file:
	line = file.read()
	json_data = json.loads(line)

data = pd.DataFrame(json_data)
data.sample(5)

Directory: ../data/23-04-21/23-04-21 Descarga Idealista.txt


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,highlight,topNewDevelopment,superTopHighlight,labels,newDevelopmentFinished
1074,99444187,https://img3.idealista.com/blur/WEB_LISTING/0/...,DS-P22497,38,4,1275000.0,flat,sale,142.0,True,...,"{'subtitle': 'Goya, Madrid', 'title': 'Piso'}",True,True,False,True,{'groupDescription': 'Top'},False,False,NaN,NaN
1058,100523144,https://img3.idealista.com/blur/WEB_LISTING/0/...,1297,20,3,615000.0,penthouse,sale,82.0,True,...,"{'subtitle': 'Prosperidad, Madrid', 'title': '...",True,True,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
1096,98895431,https://img3.idealista.com/blur/WEB_LISTING/0/...,MV4147I,44,6,1770000.0,penthouse,sale,231.0,True,...,"{'subtitle': 'Goya, Madrid', 'title': 'Ático'}",True,True,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
1166,100326995,https://img3.idealista.com/blur/WEB_LISTING/0/...,BFP-V1036,16,1,135000.0,flat,sale,40.0,False,...,"{'subtitle': 'Numancia, Madrid', 'title': 'Pis...",True,True,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
647,87679420,https://img3.idealista.com/blur/WEB_LISTING/0/...,W-02FD3W,27,NaN,495000.0,flat,sale,83.0,True,...,"{'subtitle': 'Bernabéu-Hispanoamérica, Madrid'...",True,False,False,True,{'groupDescription': 'Top'},False,False,NaN,NaN


Análizamos brevemente lo que posee, pero antes desarrollamos una función que nos ayude a agilizar el proceso.

In [11]:
def basic_dataset_info(dataset):
	rows = len(dataset)
	cols = len(dataset.columns)

	print(f"Información de dataset: {rows} registros, {cols} columnas")
	print("Revisión de columnas del dataset:")
	print("="*60)

	columns = dataset.columns
	for column in columns:
		print(f"\t-{column} ({dataset[column].dtype}):")

		nulos = dataset[column].isna().sum()
		print(f"\t\t-Registros nulos: {nulos} ({nulos/rows * 100:.3f}%)")

		cardinality = len(dataset[column].value_counts())/rows
		print(f"\t\t-Cardinalidad: {cardinality * 100:.3f}%")

Desarrollamos una función para extraer información básica del dataset.

In [12]:
basic_dataset_info(data)

Información de dataset: 1192 registros, 41 columnas
Revisión de columnas del dataset:
	-propertyCode (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 89.346%
	-thumbnail (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.591%
	-externalReference (object):
		-Registros nulos: 65 (5.453%)
		-Cardinalidad: 83.305%
	-numPhotos (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 6.963%
	-floor (object):
		-Registros nulos: 134 (11.242%)
		-Cardinalidad: 1.678%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.171%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.419%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 26.594%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.168%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.839%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinali

In [13]:
datasets.append(data)

##### Archivo: **23-04-21Datos2000(4).csv**

In [14]:
dir = f"{path}/{files[1]}"
#dir = f"{base_path}/23-04-21/{files[1]}"
print("Directory:", dir)
pd.read_csv(dir)

Directory: ../data/23-04-21/23-04-21Datos2000(4).csv


,latitude,longitude
0,40.474176,-3.718524
1,40.473012,-3.721207
2,40.469095,-3.716839
3,40.471575,-3.719830
4,40.473213,-3.720713
...,...,...
1187,40.384186,-3.671609
1188,40.384380,-3.668888
1189,40.384115,-3.670471
1190,40.382617,-3.667895


Cómo vemos, este archivo posee información que no vamos a poder asociar a ningún inmueble.

#### 20-05-2023

Dentro de la carpeta encontramos una variedad de archivos: `.txt`, `.csv` , `.xlsx`.

In [15]:
path = f"{base_path}/23-05-20"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/23-05-20
['DatosDescarga.csv', 'DatosDescarga2(1) - nuda.csv', 'DatosDescarga2(1).csv', 'Listado.txt', 'Verificaciones.xlsx', 'Verificación 2.csv']


##### Archivo: **DatosDescarga2(1).csv**

In [16]:
dir = f"{path}/{files[2]}"
print("Directory:", dir)
data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-05-20/DatosDescarga2(1).csv


,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,superTopHighlight,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
1160,1160,100529215,445000.0,flat,sale,102.0,True,2,2,barrio Adelfas,...,False,NaN,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Adelfas, Madrid",True,False
442,442,100964685,580000.0,penthouse,sale,125.0,True,3,1,barrio Cuatro Caminos,...,False,6,True,NaN,flat,penthouse,Ático,"Cuatro Caminos, Madrid",NaN,NaN
155,155,100424831,516500.0,flat,sale,80.0,True,2,2,"Calle Ronda de Segovia, 15",...,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,"Piso en Calle Ronda de Segovia, 15","Palacio, Madrid",True,True
1097,1097,101288675,1350000.0,penthouse,sale,136.0,True,3,3,Alcalá,...,False,8,True,NaN,flat,penthouse,Ático en Alcalá,"Goya, Madrid",NaN,NaN
940,940,99405784,590000.0,flat,sale,131.0,True,3,2,Paseo de la Reina Cristina,...,False,6,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Paseo de la Reina Cristina,"Jerónimos, Madrid",True,True


In [17]:
basic_dataset_info(data)

Información de dataset: 1196 registros, 37 columnas
Revisión de columnas del dataset:
	-Unnamed: 0 (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 100.000%
	-propertyCode (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.294%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.629%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.418%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 25.669%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.167%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.836%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.669%
	-address (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 49.666%
	-province (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-municipality (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.

In [18]:
datasets.append(data)

##### Archivo: **Listado.txt**

In [19]:
dir = f"{path}/{files[3]}"
print("Directory:", dir)
with open(dir, "r", encoding="utf-8") as file:
	line = file.read()
	json_data = json.loads(line)

data = pd.DataFrame(json_data)
data.sample(5)

Directory: ../data/23-05-20/Listado.txt


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,highlight,superTopHighlight,topNewDevelopment,labels,newDevelopmentFinished
301,100425420,https://img3.idealista.com/blur/WEB_LISTING/0/...,BHHS-005525,41,1,850000.0,flat,sale,123.0,True,...,"{'subtitle': 'Malasaña-Universidad, Madrid', '...",True,True,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
845,101446756,https://img3.idealista.com/blur/WEB_LISTING/0/...,ch-425543,44,5,980000.0,flat,sale,187.0,True,...,"{'subtitle': 'Bernabéu-Hispanoamérica, Madrid'...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
850,100843480,https://img3.idealista.com/blur/WEB_LISTING/0/...,ch-425429,51,4,2100000.0,flat,sale,392.0,True,...,"{'subtitle': 'Bernabéu-Hispanoamérica, Madrid'...",True,False,False,False,{'groupDescription': 'Top'},False,False,"[{'name': 'luxuryType', 'text': 'Lujo'}]",NaN
1045,101168358,https://img3.idealista.com/blur/WEB_LISTING/0/...,CLK00/UI/2023/005526,26,2,400000.0,flat,sale,83.0,False,...,"{'subtitle': 'Prosperidad, Madrid', 'title': '...",True,False,False,False,{'groupDescription': 'Top'},False,False,"[{'name': 'reformed', 'text': 'Reformado'}]",NaN
1145,100719892,https://img3.idealista.com/blur/WEB_LISTING/0/...,MOP-4296,21,NaN,769000.0,flat,sale,135.0,True,...,"{'subtitle': 'Niño Jesús, Madrid', 'title': 'P...",False,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN


In [20]:
basic_dataset_info(data)

Información de dataset: 1196 registros, 41 columnas
Revisión de columnas del dataset:
	-propertyCode (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.294%
	-thumbnail (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 87.542%
	-externalReference (object):
		-Registros nulos: 68 (5.686%)
		-Cardinalidad: 81.773%
	-numPhotos (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 7.274%
	-floor (object):
		-Registros nulos: 117 (9.783%)
		-Cardinalidad: 1.672%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.629%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.418%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 25.669%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.167%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.836%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalid

In [21]:
datasets.append(data)

##### Archivo: **Verificaciones.xlsx**

In [22]:
dir = f"{path}/{files[4]}"
print("Directory:", dir)

data = pd.read_excel(dir)
data.sample(5)

Directory: ../data/23-05-20/Verificaciones.xlsx


,Unnamed: 0,Unnamed: 1
686,100747676,2
402,99898881,4
805,100964668,2
229,98802260,2
491,100243986,2


##### Archivo: **DatosDescarga.csv**

In [23]:
dir = f"{path}/{files[0]}"
print("Directory:", dir)

data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-05-20/DatosDescarga.csv


,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,newDevelopment.1,priceByArea,detailedType,suggestedTexts,hasPlan,topNewDevelopment,superTopHighlight,floor,hasLift,parkingSpace
231,231,95197436,450000.0,flat,sale,120.0,True,3,2,Calle de Francos Rodríguez,...,False,3750.0,{'typology': 'flat'},"{'subtitle': 'Berruguete, Madrid', 'title': 'P...",True,False,False,2,True,NaN
685,685,99912724,2300000.0,penthouse,sale,370.0,True,3,4,barrio Almagro,...,False,6216.0,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Almagro, Madrid', 'title': 'Ático'}",True,False,False,6,True,NaN
1065,1065,99378233,440000.0,flat,sale,84.0,True,1,1,Corazón de María,...,False,5238.0,{'typology': 'flat'},"{'subtitle': 'Prosperidad, Madrid', 'title': '...",True,False,False,9,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
307,307,99204394,572300.0,flat,sale,123.0,True,2,2,barrio Malasaña-Universidad,...,False,4653.0,{'typology': 'flat'},"{'subtitle': 'Malasaña-Universidad, Madrid', '...",True,False,False,5,True,NaN
868,868,101082331,3190000.0,chalet,sale,268.0,False,4,2,barrio El Viso,...,False,11903.0,"{'typology': 'chalet', 'subTypology': 'semidet...","{'subtitle': 'El Viso, Madrid', 'title': 'Chal...",True,False,False,NaN,NaN,NaN


In [24]:
basic_dataset_info(data)

Información de dataset: 1196 registros, 31 columnas
Revisión de columnas del dataset:
	-Unnamed: 0 (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 100.000%
	-propertyCode (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.294%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.629%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.418%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 25.669%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.167%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.836%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.669%
	-address (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 49.666%
	-province (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-municipality (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.

In [25]:
datasets.append(data)

##### Archivo: **Verificación 2.csv**

In [26]:
dir = f"{path}/{files[5]}"
print("Directory:", dir)

data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-05-20/Verificación 2.csv


,ex,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl,n�mero de veces
620,483.0,100265984.0,860000.0,flat,sale,138.0,True,4.0,2.0,barrio Trafalgar,...,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Trafalgar, Madrid",True,True,2.0
1991,516.0,103056454.0,700000.0,flat,sale,103.0,False,3.0,1.0,barrio Trafalgar,...,5,True,NaN,flat,NaN,Piso,"Trafalgar, Madrid",NaN,NaN,1.0
1356,449.0,101388834.0,990000.0,flat,sale,178.0,True,3.0,3.0,"Calle de Orense, 22",...,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,"Piso en Calle de Orense, 22","Cuatro Caminos, Madrid",True,True,1.0
471,576.0,99776168.0,412000.0,flat,sale,118.0,True,3.0,1.0,Calle de Embajadores,...,1,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle de Embajadores,"Chopera, Madrid",True,False,1.0
1295,433.0,101309492.0,575000.0,chalet,sale,175.0,False,3.0,2.0,herrera,...,NaN,NaN,NaN,chalet,independantHouse,Casa independiente en herrera,"Valdeacederas, Madrid",NaN,NaN,1.0


In [27]:
basic_dataset_info(data)

Información de dataset: 2395 registros, 37 columnas
Revisión de columnas del dataset:
	-ex (float64):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 50.021%
	-propertyCode (float64):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 83.257%
	-price (float64):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 27.641%
	-propertyType (object):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 0.209%
	-operation (object):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 0.042%
	-size (float64):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 14.990%
	-exterior (object):
		-Registros nulos: 66 (2.756%)
		-Cardinalidad: 0.084%
	-rooms (float64):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 0.459%
	-bathrooms (float64):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 0.376%
	-address (object):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 39.499%
	-province (object):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 0.042%
	-municipality (object):
		-Registros nulos: 1 (0.042%)
		-Cardinalidad: 

In [28]:
datasets.append(data)

##### Archivo: **DatosDescarga2(1) - nuda.csv**

In [29]:
dir = f"{path}/{files[2]}"
print("Directory:", dir)

data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-05-20/DatosDescarga2(1).csv


,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,superTopHighlight,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
510,510,98249348,2000000.0,flat,sale,227.0,True,3,4,barrio Almagro,...,False,3,True,NaN,flat,NaN,Piso,"Almagro, Madrid",NaN,NaN
303,303,100740621,1390000.0,flat,sale,137.0,True,2,2,barrio Malasaña-Universidad,...,False,NaN,True,NaN,flat,NaN,Piso,"Malasaña-Universidad, Madrid",NaN,NaN
889,889,100766845,1400000.0,flat,sale,169.0,True,3,2,Calle de Castelló,...,False,5,True,NaN,flat,NaN,Piso en Calle de Castelló,"Castellana, Madrid",NaN,NaN
153,153,100424788,553500.0,flat,sale,85.0,True,2,2,"Calle Ronda de Segovia, 15",...,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,"Piso en Calle Ronda de Segovia, 15","Palacio, Madrid",True,True
529,529,100389283,875000.0,flat,sale,193.0,True,8,8,Calle de Fuencarral,...,False,3,True,NaN,flat,NaN,Piso en Calle de Fuencarral,"Chueca-Justicia, Madrid",NaN,NaN


In [30]:
basic_dataset_info(data)

Información de dataset: 1196 registros, 37 columnas
Revisión de columnas del dataset:
	-Unnamed: 0 (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 100.000%
	-propertyCode (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.294%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.629%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.418%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 25.669%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.167%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.836%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.669%
	-address (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 49.666%
	-province (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-municipality (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.

In [31]:
datasets.append(data)

#### 19-06-2023

In [ ]:

path = f"{base_path}/23-05-20"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/23-05-20
['DatosDescarga2(1).csv', 'Listado.txt', 'Verificaciones.xlsx', 'DatosDescarga.csv', 'Verificación 2.csv', 'DatosDescarga2(1) - nuda.csv']


In [33]:
path = f"{base_path}/23-06-19"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/23-06-19
['DatosDescarga.csv', 'DatosDescarga2 - nuda.csv', 'DatosDescarga2.csv', 'Listado.txt']


##### Archivo: **Listado.txt**

In [35]:
dir = f"{path}/{files[3]}"
print("Directory:", dir)
with open(dir, "r", encoding="utf-8") as file:
	line = file.read()
	json_data = json.loads(line)

data = pd.DataFrame(json_data)
data.sample(5)

Directory: ../data/23-06-19/Listado.txt


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,highlight,topNewDevelopment,superTopHighlight,labels,newDevelopmentFinished
1076,101213553,https://img3.idealista.com/blur/WEB_LISTING/0/...,as183669,34,3,1090000.0,flat,sale,175.0,True,...,"{'subtitle': 'Lista, Madrid', 'title': 'Piso e...",True,True,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
1161,101295015,https://img3.idealista.com/blur/WEB_LISTING/0/...,Venta-ERabo2h,29,1,219000.0,flat,sale,76.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
555,100735552,https://img3.idealista.com/blur/WEB_LISTING/0/...,W-02R7T1,28,NaN,385000.0,flat,sale,84.0,True,...,"{'subtitle': 'Palos de Moguer, Madrid', 'title...",True,True,False,True,{'groupDescription': 'Top'},False,False,NaN,NaN
902,101100791,https://img3.idealista.com/blur/WEB_LISTING/0/...,as184398,41,4,1365000.0,flat,sale,102.0,True,...,"{'subtitle': 'Recoletos, Madrid', 'title': 'Pi...",True,True,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
252,101657632,https://img3.idealista.com/blur/WEB_LISTING/0/...,3433-02405,70,10,695000.0,flat,sale,120.0,True,...,"{'subtitle': 'Bellas Vistas, Madrid', 'title':...",True,True,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN


In [36]:
basic_dataset_info(data)

Información de dataset: 1194 registros, 41 columnas
Revisión de columnas del dataset:
	-propertyCode (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.526%
	-thumbnail (object):
		-Registros nulos: 1 (0.084%)
		-Cardinalidad: 87.856%
	-externalReference (object):
		-Registros nulos: 74 (6.198%)
		-Cardinalidad: 82.412%
	-numPhotos (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 7.370%
	-floor (object):
		-Registros nulos: 111 (9.296%)
		-Cardinalidad: 1.759%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.693%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.419%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 26.214%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.168%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.838%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalid

In [37]:
datasets.append(data)

##### Archivo: **DatosDescarga.csv**

In [38]:
dir = f"{path}/{files[0]}"
print("Directory:", dir)

data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-06-19/DatosDescarga.csv


,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,newDevelopment.1,priceByArea,detailedType,suggestedTexts,hasPlan,topNewDevelopment,superTopHighlight,floor,hasLift,parkingSpace
562,562,101502879,469000.0,flat,sale,89.0,True,3,2,"Calle de las Peñuelas, 54",...,False,5270.0,{'typology': 'flat'},"{'subtitle': 'Acacias, Madrid', 'title': 'Piso...",True,False,False,2,True,NaN
1095,1095,99356306,750000.0,flat,sale,149.0,True,4,3,Calle del Marqués de Mondéjar,...,False,5034.0,{'typology': 'flat'},"{'subtitle': 'Fuente del Berro, Madrid', 'titl...",True,False,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
672,672,99467776,11000000.0,flat,sale,730.0,True,5,6,barrio Castellana,...,False,15068.0,{'typology': 'flat'},"{'subtitle': 'Castellana, Madrid', 'title': 'P...",False,False,False,1,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
470,470,101556702,740000.0,flat,sale,145.0,True,5,5,Calle de maudes,...,False,5103.0,{'typology': 'flat'},"{'subtitle': 'Nuevos Ministerios-Ríos Rosas, M...",False,False,False,1,False,NaN
845,845,100081532,1375000.0,flat,sale,243.0,True,4,3,barrio El Viso,...,False,5658.0,{'typology': 'flat'},"{'subtitle': 'El Viso, Madrid', 'title': 'Piso'}",True,False,False,4,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."


In [39]:
basic_dataset_info(data)

Información de dataset: 1194 registros, 31 columnas
Revisión de columnas del dataset:
	-Unnamed: 0 (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 100.000%
	-propertyCode (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.526%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.693%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.419%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 26.214%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.168%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.838%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.670%
	-address (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 47.906%
	-province (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-municipality (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.

In [40]:
datasets.append(data)

##### Archivo: **DatosDescarga2 - nuda.csv**

In [41]:
dir = f"{path}/{files[1]}"
print("Directory:", dir)

data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-06-19/DatosDescarga2 - nuda.csv


,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl,Nuda
88,88.0,100831570.0,553000.0,flat,sale,126.0,True,3.0,1.0,Calle de Fernando el Católico,...,1,True,NaN,flat,NaN,Piso en Calle de Fernando el Católico,"Gaztambide, Madrid",NaN,NaN,0.0
1166,1166.0,100086801.0,210000.0,duplex,sale,66.0,True,2.0,2.0,Calle del Monte Perdido,...,3,True,NaN,flat,duplex,Dúplex en Calle del Monte Perdido,"San Diego, Madrid",NaN,NaN,0.0
81,81.0,101578652.0,1499000.0,penthouse,sale,175.0,True,3.0,3.0,Calle de Fernández de los Ríos,...,6,True,NaN,flat,penthouse,Ático en Calle de Fernández de los Ríos,"Gaztambide, Madrid",NaN,NaN,0.0
122,122.0,100881940.0,405000.0,flat,sale,62.0,True,1.0,1.0,barrio Argüelles,...,4,True,NaN,flat,NaN,Piso,"Argüelles, Madrid",NaN,NaN,0.0
506,506.0,101184937.0,790000.0,flat,sale,110.0,True,1.0,2.0,barrio Chueca-Justicia,...,4,True,NaN,flat,NaN,Piso,"Chueca-Justicia, Madrid",NaN,NaN,0.0


In [42]:
basic_dataset_info(data)

Información de dataset: 1196 registros, 38 columnas
Revisión de columnas del dataset:
	-Unnamed: 0 (float64):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 99.833%
	-propertyCode (float64):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 88.378%
	-price (float64):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 38.629%
	-propertyType (object):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 0.418%
	-operation (object):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 26.171%
	-exterior (object):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 0.167%
	-rooms (float64):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 0.836%
	-bathrooms (float64):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 0.669%
	-address (object):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 47.826%
	-province (object):
		-Registros nulos: 2 (0.167%)
		-Cardinalidad: 0.084%
	-municipality (object):
		-Registros nulos: 2 (0.167%)
		-Cardina

In [43]:
datasets.append(data)

##### Archivo: **DatosDescarga2.csv**

In [44]:
dir = f"{path}/{files[2]}"
print("Directory:", dir)

data = pd.read_csv(dir)
data.sample(5)

Directory: ../data/23-06-19/DatosDescarga2.csv


,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,superTopHighlight,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
810,810,98886789,2700000.0,chalet,sale,350.0,False,5,6,Paseo de La Habana,...,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",chalet,terracedHouse,Chalet adosado en Paseo de La Habana,"Nueva España, Madrid",True,True
104,104,100321497,950000.0,flat,sale,249.0,True,5,3,barrio Argüelles,...,False,3,True,NaN,flat,NaN,Piso,"Argüelles, Madrid",NaN,NaN
896,896,101232479,1599000.0,flat,sale,162.0,True,3,3,Calle del Conde de Peñalver,...,False,2,True,NaN,flat,NaN,Piso en Calle del Conde de Peñalver,"Goya, Madrid",NaN,NaN
123,123,100030920,1150000.0,flat,sale,199.0,False,7,7,barrio Argüelles,...,False,NaN,True,NaN,flat,NaN,Piso,"Argüelles, Madrid",NaN,NaN
906,906,100263580,3850000.0,flat,sale,401.0,True,5,4,barrio Castellana,...,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Castellana, Madrid",True,True


In [45]:
basic_dataset_info(data)

Información de dataset: 1194 registros, 37 columnas
Revisión de columnas del dataset:
	-Unnamed: 0 (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 100.000%
	-propertyCode (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 88.526%
	-price (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 38.693%
	-propertyType (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.419%
	-operation (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-size (float64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 26.214%
	-exterior (bool):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.168%
	-rooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.838%
	-bathrooms (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.670%
	-address (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 47.906%
	-province (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.084%
	-municipality (object):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 0.

In [46]:
datasets.append(data)

#### 1) Union `21-04-2023 - 19-06-2023`
Concatenamos los todos los dataset que hemos ido guardando en `dataframes`.

In [47]:
df_union_1 = pd.concat(datasets, ignore_index=True)
df_union_1.sample(2)

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,newDevelopment.1,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl,ex,n�mero de veces,Nuda
10015,100718900,NaN,NaN,NaN,3,399000.0,flat,sale,75.0,True,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9273,101100791,https://img3.idealista.com/blur/WEB_LISTING/0/...,as184398,41.0,4,1365000.0,flat,sale,102.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 29-10-2029

In [48]:
path = f"{base_path}/23-10-29"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/23-10-29
['DatosDescarga.csv', 'DatosDescarga2.csv', 'Listado.txt']


##### Archivo: **DatosDescarga.csv**

In [49]:
df_2023_10_29 = pd.read_csv(f"{path}/DatosDescarga.csv") 
df_2023_10_29

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,newDevelopment.1,priceByArea,detailedType,suggestedTexts,hasPlan,topNewDevelopment,superTopHighlight,floor,hasLift,parkingSpace
0,0,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,False,4838.0,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Peñagrande, Madrid', 'title': 'Á...",True,False,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
1,1,102714810,370000.0,flat,sale,103.0,True,3,2,Calle Valdesangil,...,False,3592.0,{'typology': 'flat'},"{'subtitle': 'Valdezarza, Madrid', 'title': 'P...",True,False,False,1,True,NaN
2,2,102015772,385000.0,flat,sale,77.0,True,3,2,"Calle Isla Malaita, 7",...,False,5000.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,10,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
3,3,102147885,890000.0,chalet,sale,325.0,NaN,7,5,Calle de las Islas Marquesas,...,False,2738.0,"{'typology': 'chalet', 'subTypology': 'terrace...","{'subtitle': 'Peñagrande, Madrid', 'title': 'C...",True,False,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
4,4,102643242,1300000.0,chalet,sale,388.0,NaN,5,6,Calle de San Gerardo,...,False,3351.0,"{'typology': 'chalet', 'subTypology': 'indepen...","{'subtitle': 'Valdezarza, Madrid', 'title': 'C...",False,False,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1193,101654615,199999.0,flat,sale,140.0,True,3,3,Calle de Alfredo Castro Camba,...,False,1429.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",False,False,False,bj,False,NaN
1194,1194,100063977,390000.0,duplex,sale,184.0,True,4,3,Calle de Sierra Carbonera,...,False,2120.0,"{'typology': 'flat', 'subTypology': 'duplex'}","{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,False,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
1195,1195,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,False,2882.0,"{'typology': 'flat', 'subTypology': 'duplex'}","{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,False,False,bj,False,NaN
1196,1196,102963285,205000.0,flat,sale,153.0,True,3,3,Calle de Alfredo Castro Camba,...,False,1340.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",False,False,False,bj,False,NaN


##### Archivo: **DatosDescarga2.csv**

In [50]:
df_2023_10_29_2 = pd.read_csv(f"{path}/DatosDescarga2.csv")
df_2023_10_29_2

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,superTopHighlight,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
0,0,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,penthouse,Ático en Calle de Juan Sánchez,"Peñagrande, Madrid",True,True
1,1,102714810,370000.0,flat,sale,103.0,True,3,2,Calle Valdesangil,...,False,1,True,NaN,flat,NaN,Piso en Calle Valdesangil,"Valdezarza, Madrid",NaN,NaN
2,2,102015772,385000.0,flat,sale,77.0,True,3,2,"Calle Isla Malaita, 7",...,False,10,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,"Piso en Calle Isla Malaita, 7","Peñagrande, Madrid",True,True
3,3,102147885,890000.0,chalet,sale,325.0,NaN,7,5,Calle de las Islas Marquesas,...,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",chalet,terracedHouse,Chalet adosado en Calle de las Islas Marquesas,"Peñagrande, Madrid",True,True
4,4,102643242,1300000.0,chalet,sale,388.0,NaN,5,6,Calle de San Gerardo,...,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",chalet,independantHouse,Casa independiente en Calle de San Gerardo,"Valdezarza, Madrid",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1193,101654615,199999.0,flat,sale,140.0,True,3,3,Calle de Alfredo Castro Camba,...,False,bj,False,NaN,flat,NaN,Piso en Calle de Alfredo Castro Camba,"San Diego, Madrid",NaN,NaN
1194,1194,100063977,390000.0,duplex,sale,184.0,True,4,3,Calle de Sierra Carbonera,...,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,duplex,Dúplex en Calle de Sierra Carbonera,"San Diego, Madrid",True,False
1195,1195,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,False,bj,False,NaN,flat,duplex,Dúplex en Calle del Vizconde de Arlessón,"San Diego, Madrid",NaN,NaN
1196,1196,102963285,205000.0,flat,sale,153.0,True,3,3,Calle de Alfredo Castro Camba,...,False,bj,False,NaN,flat,NaN,Piso en Calle de Alfredo Castro Camba,"San Diego, Madrid",NaN,NaN


##### Archivo: **Listado.txt**

In [51]:
archivo = f"{path}/Listado.txt"

with open(archivo, "r", encoding="utf-8") as f:
    contenido = f.read()

data = json.loads(contenido) # Convertir el texto a objeto Python (lista de diccionarios)

df_listado_23_10_29 = pd.DataFrame(data)
df_listado_23_10_29

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,highlight,superTopHighlight,topNewDevelopment,newDevelopmentFinished,labels
0,100340472,https://img3.idealista.com/blur/WEB_LISTING/0/...,VM2302002,48,5,745000.0,penthouse,sale,154.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'Á...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
1,102714810,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,52,1,370000.0,flat,sale,103.0,True,...,"{'subtitle': 'Valdezarza, Madrid', 'title': 'P...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
2,102015772,https://img3.idealista.com/blur/WEB_LISTING/0/...,PV:1027,32,10,385000.0,flat,sale,77.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
3,102147885,https://img3.idealista.com/blur/WEB_LISTING/0/...,31589,59,NaN,890000.0,chalet,sale,325.0,NaN,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'C...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
4,102643242,https://img3.idealista.com/blur/WEB_LISTING/0/...,INM14861,64,NaN,1300000.0,chalet,sale,388.0,NaN,...,"{'subtitle': 'Valdezarza, Madrid', 'title': 'C...",False,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,101654615,https://img3.idealista.com/blur/WEB_LISTING/0/...,28-01485,33,bj,199999.0,flat,sale,140.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",False,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1194,100063977,https://img3.idealista.com/blur/WEB_LISTING/0/...,23185_V_ANGSC,44,2,390000.0,duplex,sale,184.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,False,True,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1195,101137233,https://img3.idealista.com/blur/WEB_LISTING/0/...,454-VIZCONDE,35,bj,98000.0,duplex,sale,34.0,False,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,True,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1196,102963285,https://img3.idealista.com/blur/WEB_LISTING/0/...,503-ACC,42,bj,205000.0,flat,sale,153.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",False,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,"[{'name': 'reformed', 'text': 'Reformado'}]"


#### 2) Union 29-10-2023

In [52]:
df_union_2 = pd.concat([df_2023_10_29, df_2023_10_29_2, df_listado_23_10_29 ], ignore_index=True)
df_union_2.sample(5)

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,numPhotos,neighborhood,distance,hasVideo,has3DTour,has360,hasStaging,highlight,newDevelopmentFinished,labels
2608,NaN,100195562,341000.0,flat,sale,71.0,True,2,1,"Calle Esperanza sanchez carrascosa, 45",...,13.0,Valdeacederas,549,False,False,False,False,{'groupDescription': 'Top'},False,NaN
2201,1003.0,102245298,830000.0,chalet,sale,180.0,NaN,5,4,Calle de Arturo Soria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510,510.0,98249348,2400000.0,flat,sale,227.0,True,3,4,barrio Almagro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
801,801.0,102882752,1560000.0,penthouse,sale,318.0,True,4,3,"Calle de la Buganvilla, 4",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2595,NaN,101474837,488500.0,flat,sale,78.0,True,1,1,Paseo de la Dirección,...,63.0,Valdeacederas,164,True,False,True,False,{'groupDescription': 'Top'},NaN,NaN


#### 01-12-2023

In [53]:
path = f"{base_path}/23-12-1"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/23-12-1
['DatosDescarga.csv', 'DatosDescarga2.csv', 'Listado.txt']


##### Archivo: **DatosDescarga.csv**

In [54]:
df_2023_12_1 = pd.read_csv(f"{path}/DatosDescarga.csv") #Mediante la función de pandas read_csv vamos a listar todos los archivos que tengamos de este tipo.
df_2023_12_1

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,newDevelopment.1,priceByArea,detailedType,suggestedTexts,hasPlan,topNewDevelopment,superTopHighlight,floor,hasLift,parkingSpace
0,0,34797321,939000.0,flat,sale,225.0,True,4,4,barrio Peñagrande,...,False,4173.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
1,1,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,False,4838.0,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Peñagrande, Madrid', 'title': 'Á...",True,False,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
2,2,102912846,466000.0,flat,sale,100.0,True,3,2,Calle de La Bañeza,...,False,4660.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
3,3,103170813,625000.0,flat,sale,116.0,True,3,2,Calle de Juan Sánchez,...,False,5388.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,1,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
4,4,102147885,890000.0,chalet,sale,325.0,NaN,7,5,Calle de las Islas Marquesas,...,False,2738.0,"{'typology': 'chalet', 'subTypology': 'terrace...","{'subtitle': 'Peñagrande, Madrid', 'title': 'C...",True,False,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,1194,96957712,250000.0,flat,sale,110.0,True,3,2,Calle Sierra Carbonera,...,False,2273.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,False,2,True,NaN
1195,1195,103269326,200000.0,flat,sale,153.0,True,3,3,Calle de Alfredo Castro Camba,...,False,1307.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,False,bj,False,NaN
1196,1196,100063977,390000.0,duplex,sale,184.0,True,4,3,Calle de Sierra Carbonera,...,False,2120.0,"{'typology': 'flat', 'subTypology': 'duplex'}","{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,False,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
1197,1197,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,False,2882.0,"{'typology': 'flat', 'subTypology': 'duplex'}","{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,False,False,bj,False,NaN


In [55]:
df_2023_12_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1199 non-null   int64  
 1   propertyCode       1199 non-null   int64  
 2   price              1199 non-null   float64
 3   propertyType       1199 non-null   object 
 4   operation          1199 non-null   object 
 5   size               1199 non-null   float64
 6   exterior           1126 non-null   object 
 7   rooms              1199 non-null   int64  
 8   bathrooms          1199 non-null   int64  
 9   address            1199 non-null   object 
 10  province           1199 non-null   object 
 11  municipality       1199 non-null   object 
 12  district           1199 non-null   object 
 13  country            1199 non-null   object 
 14  newDevelopment     1199 non-null   bool   
 15  latitude           1199 non-null   float64
 16  longitude          1199 

##### Archivo: **DatosDescarga2.csv**

In [56]:
df_2023_12_1_2= pd.read_csv(f"{path}/DatosDescarga2.csv")
df_2023_12_1_2

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,superTopHighlight,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
0,0,34797321,939000.0,flat,sale,225.0,True,4,4,barrio Peñagrande,...,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Peñagrande, Madrid",True,True
1,1,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,penthouse,Ático en Calle de Juan Sánchez,"Peñagrande, Madrid",True,True
2,2,102912846,466000.0,flat,sale,100.0,True,3,2,Calle de La Bañeza,...,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle de La Bañeza,"Peñagrande, Madrid",True,True
3,3,103170813,625000.0,flat,sale,116.0,True,3,2,Calle de Juan Sánchez,...,False,1,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle de Juan Sánchez,"Peñagrande, Madrid",True,True
4,4,102147885,890000.0,chalet,sale,325.0,NaN,7,5,Calle de las Islas Marquesas,...,False,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",chalet,terracedHouse,Chalet adosado en Calle de las Islas Marquesas,"Peñagrande, Madrid",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,1194,96957712,250000.0,flat,sale,110.0,True,3,2,Calle Sierra Carbonera,...,False,2,True,NaN,flat,NaN,Piso en Calle Sierra Carbonera,"San Diego, Madrid",NaN,NaN
1195,1195,103269326,200000.0,flat,sale,153.0,True,3,3,Calle de Alfredo Castro Camba,...,False,bj,False,NaN,flat,NaN,Piso en Calle de Alfredo Castro Camba,"San Diego, Madrid",NaN,NaN
1196,1196,100063977,390000.0,duplex,sale,184.0,True,4,3,Calle de Sierra Carbonera,...,False,2,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,duplex,Dúplex en Calle de Sierra Carbonera,"San Diego, Madrid",True,False
1197,1197,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,False,bj,False,NaN,flat,duplex,Dúplex en Calle del Vizconde de Arlessón,"San Diego, Madrid",NaN,NaN


In [57]:
df_2023_12_1_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1199 non-null   int64  
 1   propertyCode       1199 non-null   int64  
 2   price              1199 non-null   float64
 3   propertyType       1199 non-null   object 
 4   operation          1199 non-null   object 
 5   size               1199 non-null   float64
 6   exterior           1126 non-null   object 
 7   rooms              1199 non-null   int64  
 8   bathrooms          1199 non-null   int64  
 9   address            1199 non-null   object 
 10  province           1199 non-null   object 
 11  municipality       1199 non-null   object 
 12  district           1199 non-null   object 
 13  country            1199 non-null   object 
 14  newDevelopment     1199 non-null   bool   
 15  latitude           1199 non-null   float64
 16  longitude          1199 

In [58]:
df_2023_12_1_2.columns
# 'dType', 'SubdType', 'dsugTextTit', 'dsugTexsubtit','dhasParking', 'dParkingIncl' 
#En el segundo arvhivo csv se puede observar que hay nuevas columnas, tendremos que ver si las dejamos o eliminamos.

Index(['Unnamed: 0', 'propertyCode', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'district', 'country', 'newDevelopment', 'latitude',
       'longitude', 'showAddress', 'url', 'description', 'status',
       'newDevelopment.1', 'priceByArea', 'detailedType', 'suggestedTexts',
       'hasPlan', 'topNewDevelopment', 'superTopHighlight', 'floor', 'hasLift',
       'parkingSpace', 'dType', 'SubdType', 'dsugTextTit', 'dsugTexsubtit',
       'dhasParking', 'dParkingIncl'],
      dtype='object')

##### Archivo: **Listado.txt**

In [59]:
archivo = f"{path}/Listado.txt"

with open(archivo, "r", encoding="utf-8") as f:
    contenido = f.read()

data = json.loads(contenido) # Convertir el texto a objeto Python (lista de diccionarios)

df_listado_23_12_1 = pd.DataFrame(data)
df_listado_23_12_1

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,hasPlan,has3DTour,has360,hasStaging,highlight,superTopHighlight,topNewDevelopment,topPlus,newDevelopmentFinished,labels
0,34797321,https://img3.idealista.com/blur/WEB_LISTING/0/...,MIV01463 - Peña Grande,33,5,939000.0,flat,sale,225.0,True,...,True,True,False,True,{'groupDescription': 'Top'},False,False,False,NaN,NaN
1,100340472,https://img3.idealista.com/blur/WEB_LISTING/0/...,VM2302002,48,5,745000.0,penthouse,sale,154.0,True,...,True,False,True,False,{'groupDescription': 'Top'},False,False,False,NaN,NaN
2,102912846,https://img3.idealista.com/blur/WEB_LISTING/0/...,CLK00/3391.1,25,3,466000.0,flat,sale,100.0,True,...,True,False,False,False,{'groupDescription': 'Top'},False,False,False,NaN,NaN
3,103170813,https://img3.idealista.com/blur/WEB_LISTING/0/...,8351,40,1,625000.0,flat,sale,116.0,True,...,True,False,True,False,{'groupDescription': 'Top'},False,False,False,NaN,NaN
4,102147885,https://img3.idealista.com/blur/WEB_LISTING/0/...,31589,59,NaN,890000.0,chalet,sale,325.0,NaN,...,True,False,True,False,{'groupDescription': 'Top'},False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,96957712,https://img3.idealista.com/blur/WEB_LISTING/0/...,17129,21,2,250000.0,flat,sale,110.0,True,...,True,False,False,False,{'groupDescription': 'Destacado'},False,False,False,NaN,NaN
1195,103269326,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,21,bj,200000.0,flat,sale,153.0,True,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,False,NaN,NaN
1196,100063977,https://img3.idealista.com/blur/WEB_LISTING/0/...,23185_V_ANGSC,44,2,390000.0,duplex,sale,184.0,True,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,False,NaN,NaN
1197,101137233,https://img3.idealista.com/blur/WEB_LISTING/0/...,454-VIZCONDE,35,bj,98000.0,duplex,sale,34.0,False,...,True,True,False,False,{'groupDescription': 'Destacado'},False,False,False,NaN,NaN


In [60]:
df_listado_23_12_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   propertyCode            1199 non-null   object 
 1   thumbnail               1199 non-null   object 
 2   externalReference       1108 non-null   object 
 3   numPhotos               1199 non-null   int64  
 4   floor                   1094 non-null   object 
 5   price                   1199 non-null   float64
 6   propertyType            1199 non-null   object 
 7   operation               1199 non-null   object 
 8   size                    1199 non-null   float64
 9   exterior                1126 non-null   object 
 10  rooms                   1199 non-null   int64  
 11  bathrooms               1199 non-null   int64  
 12  address                 1199 non-null   object 
 13  province                1199 non-null   object 
 14  municipality            1199 non-null   

#### 01-08-2024

In [61]:
path = f"{base_path}/24-01-08"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/24-01-08
['DatosDescarga.csv', 'DatosDescarga2.csv', 'Listado.txt']


##### Archivo: **DatosDescarga.csv**

In [62]:
df_2024_01_08 = pd.read_csv(f"{path}/DatosDescarga.csv")
df_2024_01_08

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,status,newDevelopment.1,priceByArea,detailedType,suggestedTexts,hasPlan,topNewDevelopment,floor,hasLift,parkingSpace
0,0,34797321,939000.0,flat,sale,225.0,True,4,4,barrio Peñagrande,...,good,False,4173.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
1,1,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,good,False,4838.0,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Peñagrande, Madrid', 'title': 'Á...",True,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
2,2,103206000,490000.0,flat,sale,108.0,True,3,2,Calle del Valle de Mena,...,good,False,4537.0,{'typology': 'flat'},"{'subtitle': 'Valdezarza, Madrid', 'title': 'P...",True,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
3,3,102912846,480000.0,flat,sale,100.0,True,3,2,Calle de La Bañeza,...,good,False,4800.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
4,4,103523871,690000.0,flat,sale,152.0,True,3,2,barrio Peñagrande,...,good,False,4539.0,{'typology': 'flat'},"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,8,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1193,102693590,143000.0,flat,sale,110.0,True,3,1,Av. de San Diego,...,renew,False,1300.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,1,False,NaN
1194,1194,103140285,155000.0,flat,sale,52.0,True,2,1,sancho panza,...,good,False,2981.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,2,False,NaN
1195,1195,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,good,False,2882.0,"{'typology': 'flat', 'subTypology': 'duplex'}","{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,False,bj,False,NaN
1196,1196,103504021,170000.0,flat,sale,79.0,True,3,1,Calle de Santa Alicia,...,good,False,2152.0,{'typology': 'flat'},"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,2,False,NaN


##### Archivo: **DatosDescarga2.csv**

In [63]:
df_2024_01_08_2 = pd.read_csv(f"{path}/DatosDescarga2.csv")
df_2024_01_08_2

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,topNewDevelopment,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
0,0,34797321,939000.0,flat,sale,225.0,True,4,4,barrio Peñagrande,...,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Peñagrande, Madrid",True,True
1,1,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,penthouse,Ático en Calle de Juan Sánchez,"Peñagrande, Madrid",True,True
2,2,103206000,490000.0,flat,sale,108.0,True,3,2,Calle del Valle de Mena,...,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle del Valle de Mena,"Valdezarza, Madrid",True,True
3,3,102912846,480000.0,flat,sale,100.0,True,3,2,Calle de La Bañeza,...,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle de La Bañeza,"Peñagrande, Madrid",True,True
4,4,103523871,690000.0,flat,sale,152.0,True,3,2,barrio Peñagrande,...,False,8,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Peñagrande, Madrid",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1193,102693590,143000.0,flat,sale,110.0,True,3,1,Av. de San Diego,...,False,1,False,NaN,flat,NaN,Piso en Av. de San Diego,"San Diego, Madrid",NaN,NaN
1194,1194,103140285,155000.0,flat,sale,52.0,True,2,1,sancho panza,...,False,2,False,NaN,flat,NaN,Piso en sancho panza,"San Diego, Madrid",NaN,NaN
1195,1195,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,False,bj,False,NaN,flat,duplex,Dúplex en Calle del Vizconde de Arlessón,"San Diego, Madrid",NaN,NaN
1196,1196,103504021,170000.0,flat,sale,79.0,True,3,1,Calle de Santa Alicia,...,False,2,False,NaN,flat,NaN,Piso en Calle de Santa Alicia,"San Diego, Madrid",NaN,NaN


##### Archivo: **Listado.txt**

In [64]:
archivo = f"{path}/Listado.txt"

with open(archivo, "r", encoding="utf-8") as f:
    contenido = f.read()

data = json.loads(contenido) 

df_listado_24_01_08 = pd.DataFrame(data)
df_listado_24_01_08

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,highlight,topNewDevelopment,topPlus,newDevelopmentFinished,labels
0,34797321,https://img3.idealista.com/blur/WEB_LISTING/0/...,MIV01463 - Peña Grande,33,5,939000.0,flat,sale,225.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,True,False,True,{'groupDescription': 'Top'},False,False,NaN,NaN
1,100340472,https://img3.idealista.com/blur/WEB_LISTING/0/...,VM2302002,48,5,745000.0,penthouse,sale,154.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'Á...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
2,103206000,https://img3.idealista.com/blur/WEB_LISTING/0/...,CLK00/3561.1,46,3,490000.0,flat,sale,108.0,True,...,"{'subtitle': 'Valdezarza, Madrid', 'title': 'P...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
3,102912846,https://img3.idealista.com/blur/WEB_LISTING/0/...,CLK00/3391.1,25,3,480000.0,flat,sale,100.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
4,103523871,https://img3.idealista.com/blur/WEB_LISTING/0/...,MIV0986,66,8,690000.0,flat,sale,152.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,102693590,https://img3.idealista.com/blur/WEB_LISTING/0/...,PUERTO DE ARLABAN,34,1,143000.0,flat,sale,110.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1194,103140285,https://img3.idealista.com/blur/WEB_LISTING/0/...,MA-565,29,2,155000.0,flat,sale,52.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,True,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1195,101137233,https://img3.idealista.com/blur/WEB_LISTING/0/...,454-VIZCONDE,35,bj,98000.0,duplex,sale,34.0,False,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Dú...",True,True,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1196,103504021,https://img3.idealista.com/blur/WEB_LISTING/0/...,19845,18,2,170000.0,flat,sale,79.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN


#### 3) Union `01-12-2023` - `01-08-2024`

In [65]:
df_union_3 = pd.concat([df_2023_12_1, df_2023_12_1_2, df_listado_23_12_1,df_2024_01_08, df_2024_01_08_2, df_listado_24_01_08 ], ignore_index=True)
df_union_3.sample(5)
# El concat sirve para que tengamos un solo Datframe unificado 
# Además algunas columnas tienen los mismo datos que otras pero en formato diccionario (parking space- dhasparking).

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,neighborhood,distance,hasVideo,has3DTour,has360,hasStaging,highlight,topPlus,newDevelopmentFinished,labels
2555,NaN,99278025,199310.0,flat,sale,43.0,NaN,1,1,Calle de la Ventosa,...,Palacio,323,False,False,True,False,{'groupDescription': 'Top'},False,NaN,NaN
2850,NaN,102776186,526500.0,penthouse,sale,84.0,True,2,2,"Calle Mariano Fernández, 6",...,Berruguete,456,False,False,False,False,{'groupDescription': 'Top'},False,False,NaN
7095,NaN,102852963,1681500.0,penthouse,sale,146.0,True,2,3,barrio Goya,...,Goya,503,True,True,False,False,{'groupDescription': 'Top'},False,NaN,NaN
7136,NaN,102943291,700000.0,flat,sale,107.0,True,3,2,Calle de Ángel Ganivet,...,Niño Jesús,308,True,False,False,False,{'groupDescription': 'Top'},False,NaN,NaN
5317,522.0,102751017,389000.0,flat,sale,69.0,False,2,2,aduana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_union_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7191 entries, 0 to 7190
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              4794 non-null   float64
 1   propertyCode            7191 non-null   object 
 2   price                   7191 non-null   float64
 3   propertyType            7191 non-null   object 
 4   operation               7191 non-null   object 
 5   size                    7191 non-null   float64
 6   exterior                6777 non-null   object 
 7   rooms                   7191 non-null   int64  
 8   bathrooms               7191 non-null   int64  
 9   address                 7191 non-null   object 
 10  province                7191 non-null   object 
 11  municipality            7191 non-null   object 
 12  district                7191 non-null   object 
 13  country                 7191 non-null   object 
 14  newDevelopment          7191 non-null   

In [67]:
df_union_3.columns

Index(['Unnamed: 0', 'propertyCode', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'district', 'country', 'newDevelopment', 'latitude',
       'longitude', 'showAddress', 'url', 'description', 'status',
       'newDevelopment.1', 'priceByArea', 'detailedType', 'suggestedTexts',
       'hasPlan', 'topNewDevelopment', 'superTopHighlight', 'floor', 'hasLift',
       'parkingSpace', 'dType', 'SubdType', 'dsugTextTit', 'dsugTexsubtit',
       'dhasParking', 'dParkingIncl', 'thumbnail', 'externalReference',
       'numPhotos', 'neighborhood', 'distance', 'hasVideo', 'has3DTour',
       'has360', 'hasStaging', 'highlight', 'topPlus',
       'newDevelopmentFinished', 'labels'],
      dtype='object')

In [68]:
duplicados = df_union_3["propertyCode"].duplicated()
df_union_3.loc[duplicados]
# Se puede observar que hay 4267 filas duplicadas las cuales tendremos que eliminar o dejar.

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,neighborhood,distance,hasVideo,has3DTour,has360,hasStaging,highlight,topPlus,newDevelopmentFinished,labels
87,87.0,101466187,1100000.0,flat,sale,227.0,True,5,4,barrio Vallehermoso,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,91.0,101135248,745000.0,flat,sale,153.0,False,4,3,Paseo de San Francisco de Sales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,105.0,102849306,824000.0,flat,sale,159.0,True,7,3,Calle tutor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,111.0,92677130,660000.0,penthouse,sale,58.0,True,1,1,Calle de la Princesa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,147.0,102612566,395000.0,flat,sale,97.0,True,3,1,Calle de Juan Duque,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7181,NaN,101466120,370000.0,penthouse,sale,138.0,True,3,2,Calle de los Montes Pirineos,...,Palomeras Bajas,255,False,False,True,False,{'groupDescription': 'Destacado'},False,NaN,NaN
7186,NaN,102693590,143000.0,flat,sale,110.0,True,3,1,Av. de San Diego,...,San Diego,264,True,False,False,False,{'groupDescription': 'Destacado'},False,NaN,NaN
7187,NaN,103140285,155000.0,flat,sale,52.0,True,2,1,sancho panza,...,San Diego,322,True,True,False,False,{'groupDescription': 'Destacado'},False,NaN,NaN
7188,NaN,101137233,98000.0,duplex,sale,34.0,False,2,1,Calle del Vizconde de Arlessón,...,San Diego,435,True,True,False,False,{'groupDescription': 'Destacado'},False,NaN,NaN


In [69]:
df_union_3.isna().sum().sort_values(ascending=False)
#ParkingIncl
#dhasparking eliminarla y dejar parkingspace
#SubType eliminar y dejar propertyType
#Eliminar labels y dejar hasPlan
#externalReference no nos aporta información útil

newDevelopmentFinished    7104
labels                    6946
SubdType                  6782
dhasParking               6323
dParkingIncl              6323
externalReference         4974
highlight                 4924
distance                  4794
thumbnail                 4794
dsugTexsubtit             4794
dType                     4794
neighborhood              4794
numPhotos                 4794
hasVideo                  4794
hasStaging                4794
dsugTextTit               4794
has360                    4794
has3DTour                 4794
topPlus                   4794
parkingSpace              4587
superTopHighlight         3594
Unnamed: 0                2397
newDevelopment.1          2397
floor                      609
exterior                   414
hasLift                    297
address                      0
bathrooms                    0
rooms                        0
size                         0
propertyType                 0
operation                    0
price   

In [70]:
df_union_3["SubdType"].value_counts()
#propertyType

SubdType
penthouse            201
duplex                85
independantHouse      45
terracedHouse         32
studio                28
semidetachedHouse     18
Name: count, dtype: int64

In [71]:
df_union_3["labels"].value_counts()

labels
[{'name': 'luxuryType', 'text': 'Lujo'}]                                            143
[{'name': 'apartamentoType', 'text': 'Apartamento'}]                                 56
[{'name': 'bright', 'text': 'Luminoso'}]                                             14
[{'name': 'luxuryType', 'text': 'Lujo'}, {'name': 'villaType', 'text': 'Villa'}]      8
[{'name': 'forInvestors', 'text': 'Para inversores'}]                                 7
[{'name': 'withViews', 'text': 'Con vistas'}]                                         6
[{'name': 'reformed', 'text': 'Reformado'}]                                           6
[{'name': 'atticStudioType', 'text': 'Buhardilla'}]                                   3
[{'name': 'villaType', 'text': 'Villa'}]                                              2
Name: count, dtype: int64

In [72]:
df_union_3["externalReference"].value_counts()

externalReference
LCM24099      7
Bajo A        6
LCM26163      5
AR191452      4
vp188223      4
             ..
793           1
CLK00/3289    1
CLK00/3373    1
1-1396        1
MV4815I       1
Name: count, Length: 1331, dtype: int64

In [73]:
df_union_3["highlight"].value_counts()

highlight
{'groupDescription': 'Top'}          1685
{'groupDescription': 'Destacado'}     582
Name: count, dtype: int64

In [74]:
df_union_3["dType"].value_counts()

dType
flat      2298
chalet      99
Name: count, dtype: int64

#### 11-03-2024
##### Archivo: **DatosDescarga2.csv**

In [75]:
path = f"{base_path}/24-03-11"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/24-03-11
['DatosDescarga.csv', 'DatosDescarga2.csv', 'Listado.txt']


In [76]:
df_2024_03_11 = pd.read_csv(f"{path}/DatosDescarga2.csv")
df_2024_03_11

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,topNewDevelopment,floor,hasLift,parkingSpace,dType,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl
0,0,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,False,5,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,penthouse,Ático en Calle de Juan Sánchez,"Peñagrande, Madrid",True,True
1,1,103523871,675000.0,flat,sale,152.0,True,3,2,barrio Peñagrande,...,False,8,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso,"Peñagrande, Madrid",True,True
2,2,104003668,675000.0,flat,sale,151.0,True,3,2,Calle de Isla Cristina,...,False,bj,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle de Isla Cristina,"Valdezarza, Madrid",True,True
3,3,102912846,480000.0,flat,sale,100.0,True,3,2,Calle de La Bañeza,...,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle de La Bañeza,"Peñagrande, Madrid",True,True
4,4,103206000,490000.0,flat,sale,108.0,True,3,2,Calle del Valle de Mena,...,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",flat,NaN,Piso en Calle del Valle de Mena,"Valdezarza, Madrid",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,1191,98311246,205265.0,flat,sale,98.0,True,3,2,Calle Sierra Carbonera,...,False,bj,False,NaN,flat,NaN,Piso en Calle Sierra Carbonera,"Palomeras Bajas, Madrid",NaN,NaN
1192,1192,102097920,165000.0,flat,sale,60.0,True,1,1,"Calle Carlos Martín Álvarez , 59",...,False,2,True,NaN,flat,NaN,"Piso en Calle Carlos Martín Álvarez , 59","San Diego, Madrid",NaN,NaN
1193,1193,102097872,170000.0,flat,sale,60.0,True,1,1,"Calle Carlos Martín Álvarez , 59",...,False,1,True,NaN,flat,NaN,"Piso en Calle Carlos Martín Álvarez , 59","San Diego, Madrid",NaN,NaN
1194,1194,104059214,115000.0,flat,sale,39.0,True,1,1,Calle Avelino Fernández de la Poza,...,False,2,True,NaN,flat,NaN,Piso en Calle Avelino Fernández de la Poza,"Entrevías, Madrid",NaN,NaN


In [77]:
df_2024_03_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1196 non-null   int64  
 1   propertyCode       1196 non-null   int64  
 2   price              1196 non-null   float64
 3   propertyType       1196 non-null   object 
 4   operation          1196 non-null   object 
 5   size               1196 non-null   float64
 6   exterior           1125 non-null   object 
 7   rooms              1196 non-null   int64  
 8   bathrooms          1196 non-null   int64  
 9   address            1196 non-null   object 
 10  province           1196 non-null   object 
 11  municipality       1196 non-null   object 
 12  district           1196 non-null   object 
 13  country            1196 non-null   object 
 14  newDevelopment     1196 non-null   bool   
 15  latitude           1196 non-null   float64
 16  longitude          1196 

In [78]:
df_2024_03_11.columns  # Aquí quiero saber el nombre y número de columnas que tengo en el documento CSV

Index(['Unnamed: 0', 'propertyCode', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'district', 'country', 'newDevelopment', 'latitude',
       'longitude', 'showAddress', 'url', 'description', 'status',
       'newDevelopment.1', 'priceByArea', 'detailedType', 'suggestedTexts',
       'hasPlan', 'topNewDevelopment', 'floor', 'hasLift', 'parkingSpace',
       'dType', 'SubdType', 'dsugTextTit', 'dsugTexsubtit', 'dhasParking',
       'dParkingIncl'],
      dtype='object')

In [79]:
df_2024_03_11.value_counts()

Unnamed: 0  propertyCode  price      propertyType  operation  size   exterior  rooms  bathrooms  address                                province  municipality  district             country  newDevelopment  latitude   longitude  showAddress  url                                             description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

##### Archivo: **Listado.txt**

In [80]:
archivo_txt_2024_03_11 = f"{path}/Listado.txt"

with open(archivo_txt_2024_03_11, "r", encoding="utf-8") as f:
    datos_txt = f.read()

datos = json.loads(datos_txt) # Convertir el texto a objeto Python (lista de diccionarios)

df_archivo_txt_2024_03_11 = pd.DataFrame(datos)
df_archivo_txt_2024_03_11

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,highlight,topNewDevelopment,topPlus,labels,newDevelopmentFinished
0,100340472,https://img3.idealista.com/blur/WEB_LISTING/0/...,VM2302002,48,5,745000.0,penthouse,sale,154.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'Á...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
1,103523871,https://img3.idealista.com/blur/WEB_LISTING/0/...,MIV01986,69,8,675000.0,flat,sale,152.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,True,True,{'groupDescription': 'Top'},False,False,NaN,NaN
2,104003668,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,45,bj,675000.0,flat,sale,151.0,True,...,"{'subtitle': 'Valdezarza, Madrid', 'title': 'P...",True,False,True,False,{'groupDescription': 'Top'},False,False,NaN,NaN
3,102912846,https://img3.idealista.com/blur/WEB_LISTING/0/...,CLK00/3391.1,30,3,480000.0,flat,sale,100.0,True,...,"{'subtitle': 'Peñagrande, Madrid', 'title': 'P...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
4,103206000,https://img3.idealista.com/blur/WEB_LISTING/0/...,CLK00/3561.1,29,3,490000.0,flat,sale,108.0,True,...,"{'subtitle': 'Valdezarza, Madrid', 'title': 'P...",True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,98311246,https://img3.idealista.com/blur/WEB_LISTING/0/...,CCI-001-000041801,18,bj,205265.0,flat,sale,98.0,True,...,"{'subtitle': 'Palomeras Bajas, Madrid', 'title...",True,False,True,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN
1192,102097920,https://img3.idealista.com/blur/WEB_LISTING/0/...,2º,6,2,165000.0,flat,sale,60.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,False
1193,102097872,https://img3.idealista.com/blur/WEB_LISTING/0/...,1º,6,1,170000.0,flat,sale,60.0,True,...,"{'subtitle': 'San Diego, Madrid', 'title': 'Pi...",True,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,False
1194,104059214,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,15,2,115000.0,flat,sale,39.0,True,...,"{'subtitle': 'Entrevías, Madrid', 'title': 'Pi...",False,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN


In [81]:
df_archivo_txt_2024_03_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   propertyCode            1196 non-null   object 
 1   thumbnail               1196 non-null   object 
 2   externalReference       1091 non-null   object 
 3   numPhotos               1196 non-null   int64  
 4   floor                   1088 non-null   object 
 5   price                   1196 non-null   float64
 6   propertyType            1196 non-null   object 
 7   operation               1196 non-null   object 
 8   size                    1196 non-null   float64
 9   exterior                1125 non-null   object 
 10  rooms                   1196 non-null   int64  
 11  bathrooms               1196 non-null   int64  
 12  address                 1196 non-null   object 
 13  province                1196 non-null   object 
 14  municipality            1196 non-null   

In [82]:
df_archivo_txt_2024_03_11.columns    # Aquí quiero ver el número y nombre de las columnas que tiene el txt, para compararlo con el archivo csv, por si son iguales o tienen algun tipo de relación

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor',
       'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms',
       'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'distance', 'description', 'hasVideo', 'status',
       'newDevelopment', 'hasLift', 'parkingSpace', 'priceByArea',
       'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360',
       'hasStaging', 'highlight', 'topNewDevelopment', 'topPlus', 'labels',
       'newDevelopmentFinished'],
      dtype='object')

#### 20-04-2024

In [83]:
path = f"{base_path}/24-04-20"
print("Directory:", path)
files = os.listdir(path)
print(files)

Directory: ../data/24-04-20
['DatosDescarga.csv', 'DatosDescarga2.csv', 'Listado.txt']


##### Archivo: **DatosDescarga.csv** - **DatosDescarga2.csv**

In [84]:
df_2024_04_20 = [pd.read_csv(f"{path}/DatosDescarga.csv"),
                 pd.read_csv(f"{path}/DatosDescarga2.csv")]
            # Aqui he cargado varios archivos csv en una variable, ya que se encontraban dentro de la misma carpeta y compartían la fecha)
df_2024_04_20

[      Unnamed: 0  propertyCode     price propertyType operation   size  \
 0              0     104204737  385000.0         flat      sale   92.0   
 1              1     103138983  545000.0         flat      sale  118.0   
 2              2     103944798  214000.0         flat      sale   56.0   
 3              3     104655895  209900.0         flat      sale   47.0   
 4              4     104003668  675000.0         flat      sale  151.0   
 ...          ...           ...       ...          ...       ...    ...   
 1193        1193      98311246  205265.0         flat      sale   98.0   
 1194        1194     103269326  195000.0         flat      sale  153.0   
 1195        1195     104611644  155800.0         flat      sale   52.0   
 1196        1196     104173949  135000.0       chalet      sale   64.0   
 1197        1197      99728040  155000.0         flat      sale   79.0   
 
      exterior  rooms  bathrooms                        address  ... status  \
 0        True     

In [85]:
df_2024_04_20[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1198 non-null   int64  
 1   propertyCode       1198 non-null   int64  
 2   price              1198 non-null   float64
 3   propertyType       1198 non-null   object 
 4   operation          1198 non-null   object 
 5   size               1198 non-null   float64
 6   exterior           1117 non-null   object 
 7   rooms              1198 non-null   int64  
 8   bathrooms          1198 non-null   int64  
 9   address            1198 non-null   object 
 10  province           1198 non-null   object 
 11  municipality       1198 non-null   object 
 12  district           1198 non-null   object 
 13  country            1198 non-null   object 
 14  newDevelopment     1198 non-null   bool   
 15  latitude           1198 non-null   float64
 16  longitude          1198 

In [86]:
df_2024_04_20[0].columns  # Aquí quiero saber el nombre y número de columnas que tengo en el documento CSV

Index(['Unnamed: 0', 'propertyCode', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'district', 'country', 'newDevelopment', 'latitude',
       'longitude', 'showAddress', 'url', 'description', 'status',
       'newDevelopment.1', 'priceByArea', 'detailedType', 'suggestedTexts',
       'hasPlan', 'topNewDevelopment', 'floor', 'hasLift', 'parkingSpace'],
      dtype='object')

In [87]:
df_2024_04_20[0].value_counts()

Unnamed: 0  propertyCode  price     propertyType  operation  size   exterior  rooms  bathrooms  address                                province  municipality  district            country  newDevelopment  latitude   longitude  showAddress  url                                            description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

##### Archivo: **Listado.txt**

In [88]:
archivo_txt_2024_04_20 = f"{path}/Listado.txt"

with open(archivo_txt_2024_04_20, "r", encoding="utf-8") as f:
    datos_txt = f.read()

datos = json.loads(datos_txt) # Convertir el texto a objeto Python (lista de diccionarios)

df_archivo_txt_2024_04_20 = pd.DataFrame(datos)
df_archivo_txt_2024_04_20

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,priceInfo,propertyType,operation,size,...,hasPlan,has3DTour,has360,hasStaging,highlight,topNewDevelopment,topPlus,parkingSpace,labels,newDevelopmentFinished
0,104204737,https://img3.idealista.com/blur/WEB_LISTING/0/...,RM-2035,22,11,385000.0,{'price': {'amount': 385000.0}},flat,sale,92.0,...,True,False,False,False,{'groupDescription': 'Top'},False,False,NaN,NaN,NaN
1,103138983,https://img3.idealista.com/blur/WEB_LISTING/0/...,040091,24,2,545000.0,{'price': {'amount': 545000.0}},flat,sale,118.0,...,True,False,True,False,{'groupDescription': 'Top'},False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...","[{'name': 'reformed', 'text': 'Reformado'}]",NaN
2,103944798,https://img3.idealista.com/blur/WEB_LISTING/0/...,8309,37,6,214000.0,{'price': {'amount': 214000.0}},flat,sale,56.0,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN,NaN
3,104655895,https://img3.idealista.com/blur/WEB_LISTING/0/...,VO200424,31,bj,209900.0,{'price': {'amount': 209900.0}},flat,sale,47.0,...,True,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN,NaN
4,104003668,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,45,bj,675000.0,{'price': {'amount': 675000.0}},flat,sale,151.0,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,98311246,https://img3.idealista.com/blur/WEB_LISTING/0/...,CCI-001-000041801,18,bj,205265.0,{'price': {'amount': 205265.0}},flat,sale,98.0,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN,NaN
1194,103269326,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,21,bj,195000.0,{'price': {'amount': 195000.0}},flat,sale,153.0,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,NaN,NaN,NaN
1195,104611644,https://img3.idealista.com/blur/WEB_LISTING/0/...,5183,13,NaN,155800.0,{'price': {'amount': 155800.0}},flat,sale,52.0,...,True,False,False,False,{'groupDescription': 'Destacado'},False,False,NaN,"[{'name': 'apartamentoType', 'text': 'Apartame...",NaN
1196,104173949,https://img3.idealista.com/blur/WEB_LISTING/0/...,4091,24,NaN,135000.0,{'price': {'amount': 135000.0}},chalet,sale,64.0,...,True,False,True,False,{'groupDescription': 'Destacado'},False,False,NaN,"[{'name': 'casaBajaType', 'text': 'Casa baja'}]",NaN


In [89]:
df_archivo_txt_2024_04_20.info()  #sacamos la información que contiene el txt

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   propertyCode            1198 non-null   object 
 1   thumbnail               1198 non-null   object 
 2   externalReference       1093 non-null   object 
 3   numPhotos               1198 non-null   int64  
 4   floor                   1070 non-null   object 
 5   price                   1198 non-null   float64
 6   priceInfo               1198 non-null   object 
 7   propertyType            1198 non-null   object 
 8   operation               1198 non-null   object 
 9   size                    1198 non-null   float64
 10  exterior                1117 non-null   object 
 11  rooms                   1198 non-null   int64  
 12  bathrooms               1198 non-null   int64  
 13  address                 1198 non-null   object 
 14  province                1198 non-null   

In [90]:
df_archivo_txt_2024_04_20.columns    # Aquí quiero ver el número y nombre de las columnas que tiene el txt, para compararlo con el archivo csv, por si son iguales o tienen algun tipo de relación

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor',
       'price', 'priceInfo', 'propertyType', 'operation', 'size', 'exterior',
       'rooms', 'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'distance', 'description', 'hasVideo', 'status',
       'newDevelopment', 'hasLift', 'priceByArea', 'detailedType',
       'suggestedTexts', 'hasPlan', 'has3DTour', 'has360', 'hasStaging',
       'highlight', 'topNewDevelopment', 'topPlus', 'parkingSpace', 'labels',
       'newDevelopmentFinished'],
      dtype='object')

#### 4) Union `11-03-2024` - `20-04-2024`

In [91]:
df_union_4 = pd.concat([df_2024_03_11, df_archivo_txt_2024_03_11, df_2024_04_20[0], df_2024_04_20[1], df_archivo_txt_2024_04_20 ], ignore_index=True)

df_union_4

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,distance,hasVideo,has3DTour,has360,hasStaging,highlight,topPlus,labels,newDevelopmentFinished,priceInfo
0,0.0,100340472,745000.0,penthouse,sale,154.0,True,4,3,Calle de Juan Sánchez,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,103523871,675000.0,flat,sale,152.0,True,3,2,barrio Peñagrande,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,104003668,675000.0,flat,sale,151.0,True,3,2,Calle de Isla Cristina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,102912846,480000.0,flat,sale,100.0,True,3,2,Calle de La Bañeza,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,103206000,490000.0,flat,sale,108.0,True,3,2,Calle del Valle de Mena,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5981,NaN,98311246,205265.0,flat,sale,98.0,True,3,2,calle Sierra Carbonera,...,503,False,False,True,False,{'groupDescription': 'Destacado'},False,NaN,NaN,{'price': {'amount': 205265.0}}
5982,NaN,103269326,195000.0,flat,sale,153.0,True,3,3,calle de Alfredo Castro Camba,...,319,True,False,True,False,{'groupDescription': 'Destacado'},False,NaN,NaN,{'price': {'amount': 195000.0}}
5983,NaN,104611644,155800.0,flat,sale,52.0,True,1,1,barrio San Diego,...,236,False,False,False,False,{'groupDescription': 'Destacado'},False,"[{'name': 'apartamentoType', 'text': 'Apartame...",NaN,{'price': {'amount': 155800.0}}
5984,NaN,104173949,135000.0,chalet,sale,64.0,NaN,2,1,calle de Gregorio Navas,...,358,True,False,True,False,{'groupDescription': 'Destacado'},False,"[{'name': 'casaBajaType', 'text': 'Casa baja'}]",NaN,{'price': {'amount': 135000.0}}


In [92]:
df_union_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5986 entries, 0 to 5985
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              3592 non-null   float64
 1   propertyCode            5986 non-null   object 
 2   price                   5986 non-null   float64
 3   propertyType            5986 non-null   object 
 4   operation               5986 non-null   object 
 5   size                    5986 non-null   float64
 6   exterior                5601 non-null   object 
 7   rooms                   5986 non-null   int64  
 8   bathrooms               5986 non-null   int64  
 9   address                 5986 non-null   object 
 10  province                5986 non-null   object 
 11  municipality            5986 non-null   object 
 12  district                5986 non-null   object 
 13  country                 5986 non-null   object 
 14  newDevelopment          5986 non-null   

In [93]:
df_union_4.columns

Index(['Unnamed: 0', 'propertyCode', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'district', 'country', 'newDevelopment', 'latitude',
       'longitude', 'showAddress', 'url', 'description', 'status',
       'newDevelopment.1', 'priceByArea', 'detailedType', 'suggestedTexts',
       'hasPlan', 'topNewDevelopment', 'floor', 'hasLift', 'parkingSpace',
       'dType', 'SubdType', 'dsugTextTit', 'dsugTexsubtit', 'dhasParking',
       'dParkingIncl', 'thumbnail', 'externalReference', 'numPhotos',
       'neighborhood', 'distance', 'hasVideo', 'has3DTour', 'has360',
       'hasStaging', 'highlight', 'topPlus', 'labels',
       'newDevelopmentFinished', 'priceInfo'],
      dtype='object')

In [94]:
duplicados_2 = df_union_4["propertyCode"].duplicated()
df_union_4.loc[duplicados_2]

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,distance,hasVideo,has3DTour,has360,hasStaging,highlight,topPlus,labels,newDevelopmentFinished,priceInfo
21,21.0,104204737,385000.0,flat,sale,92.0,True,3,2,Calle de Valderrodrigo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,82.0,104021671,1200000.0,flat,sale,229.0,True,5,3,Paseo de san francisco de sales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,89.0,101135248,745000.0,flat,sale,153.0,False,4,3,Paseo de San Francisco de Sales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,118.0,104138754,349000.0,flat,sale,51.0,True,1,1,barrio Argüelles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,143.0,102662115,245000.0,flat,sale,70.0,True,1,1,Calle de Manzanares,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5979,NaN,102129807,168000.0,flat,sale,71.0,True,3,1,calle de Julia Mediavilla,...,266,False,False,False,False,{'groupDescription': 'Destacado'},False,NaN,NaN,{'price': {'amount': 168000.0}}
5981,NaN,98311246,205265.0,flat,sale,98.0,True,3,2,calle Sierra Carbonera,...,503,False,False,True,False,{'groupDescription': 'Destacado'},False,NaN,NaN,{'price': {'amount': 205265.0}}
5982,NaN,103269326,195000.0,flat,sale,153.0,True,3,3,calle de Alfredo Castro Camba,...,319,True,False,True,False,{'groupDescription': 'Destacado'},False,NaN,NaN,{'price': {'amount': 195000.0}}
5984,NaN,104173949,135000.0,chalet,sale,64.0,NaN,2,1,calle de Gregorio Navas,...,358,True,False,True,False,{'groupDescription': 'Destacado'},False,"[{'name': 'casaBajaType', 'text': 'Casa baja'}]",NaN,{'price': {'amount': 135000.0}}


### Unificación de datasets

Unificamos todos los datasets que hemos estado construyendo `df_union_1`, `df_union_2`, `df_union_3`, `df_union_4`, en uno general para realizar el análisis y la exploración de los datos.

In [95]:
df_raw = pd.concat([df_union_1, df_union_2, df_union_3, df_union_4],
				   ignore_index=True)
df_raw

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,SubdType,dsugTextTit,dsugTexsubtit,dhasParking,dParkingIncl,ex,n�mero de veces,Nuda,topPlus,priceInfo
0,97392656,https://img3.idealista.com/blur/WEB_LISTING/0/...,MIV01885,36.0,4,479000.0,flat,sale,84.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100340472,https://img3.idealista.com/blur/WEB_LISTING/0/...,VM2302002,46.0,5,760000.0,penthouse,sale,154.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100070468,https://img3.idealista.com/blur/WEB_LISTING/0/...,MIV01915,19.0,1,780000.0,flat,sale,207.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101165882,https://img3.idealista.com/blur/WEB_LISTING/0/...,La Serna del Monte,44.0,1,409000.0,flat,sale,126.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101087189,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,46.0,bj,409000.0,flat,sale,126.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29915,98311246,https://img3.idealista.com/blur/WEB_LISTING/0/...,CCI-001-000041801,18.0,bj,205265.0,flat,sale,98.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'price': {'amount': 205265.0}}
29916,103269326,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,21.0,bj,195000.0,flat,sale,153.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'price': {'amount': 195000.0}}
29917,104611644,https://img3.idealista.com/blur/WEB_LISTING/0/...,5183,13.0,NaN,155800.0,flat,sale,52.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'price': {'amount': 155800.0}}
29918,104173949,https://img3.idealista.com/blur/WEB_LISTING/0/...,4091,24.0,NaN,135000.0,chalet,sale,64.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'price': {'amount': 135000.0}}


El dataset unificado cuenta con `29922 registros y 54 columnas`. A lo largo estuvimos viendo que existen muchos registros duplicados y columnas que merecen un tratamiento especial que necesitamos depurar.

#### Checkpoint: Guardamos los datos.

In [ ]:
#df_raw.to_csv(f"{base_path}/df_raw.csv", sep=",")

## EDA

> **Hipótesis**
>
> ¿La planta influye en el precio de la vivienda?
>
> El precio de las propiedades en venta en Madrid es más alto en las zonas cercanas al centro de la ciudad que en las zonas periféricas, incluso controlando el tamaño y las características de las propiedades.
>
> Las viviendas situadas en plantas altas son más caras si el edificio tiene ascensor.
>
> La ausencia de ascensor reduce significativamente el precio en pisos altos.

In [109]:
#df_raw = pd.read_csv(filepath_or_buffer=f"{base_path}/df_raw")

In [108]:
df_raw = pd.read_csv(f"{base_path}/df_raw.csv", low_memory=False)

In [110]:
basic_dataset_info(df_raw)

Información de dataset: 29922 registros, 55 columnas
Revisión de columnas del dataset:
	-Unnamed: 0.1 (int64):
		-Registros nulos: 0 (0.000%)
		-Cardinalidad: 100.000%
	-propertyCode (float64):
		-Registros nulos: 5 (0.017%)
		-Cardinalidad: 14.902%
	-thumbnail (object):
		-Registros nulos: 20353 (68.020%)
		-Cardinalidad: 18.107%
	-externalReference (object):
		-Registros nulos: 21031 (70.286%)
		-Cardinalidad: 13.371%
	-numPhotos (float64):
		-Registros nulos: 20351 (68.014%)
		-Cardinalidad: 0.341%
	-floor (object):
		-Registros nulos: 2781 (9.294%)
		-Cardinalidad: 0.084%
	-price (float64):
		-Registros nulos: 5 (0.017%)
		-Cardinalidad: 3.586%
	-propertyType (object):
		-Registros nulos: 5 (0.017%)
		-Cardinalidad: 0.017%
	-operation (object):
		-Registros nulos: 5 (0.017%)
		-Cardinalidad: 0.003%
	-size (float64):
		-Registros nulos: 5 (0.017%)
		-Cardinalidad: 1.390%
	-exterior (object):
		-Registros nulos: 1049 (3.506%)
		-Cardinalidad: 0.007%
	-rooms (float64):
		-Registros nu

### Columnas

In [111]:
df_raw.columns

Index(['Unnamed: 0.1', 'propertyCode', 'thumbnail', 'externalReference',
       'numPhotos', 'floor', 'price', 'propertyType', 'operation', 'size',
       'exterior', 'rooms', 'bathrooms', 'address', 'province', 'municipality',
       'district', 'country', 'neighborhood', 'latitude', 'longitude',
       'showAddress', 'url', 'distance', 'description', 'hasVideo', 'status',
       'newDevelopment', 'hasLift', 'parkingSpace', 'priceByArea',
       'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360',
       'hasStaging', 'highlight', 'topNewDevelopment', 'superTopHighlight',
       'labels', 'newDevelopmentFinished', 'Unnamed: 0', 'newDevelopment.1',
       'dType', 'SubdType', 'dsugTextTit', 'dsugTexsubtit', 'dhasParking',
       'dParkingIncl', 'ex', 'n�mero de veces', 'Nuda', 'topPlus',
       'priceInfo'],
      dtype='object')

Eliminamos las que a simple vista no aportan valor o la info se encuentra en otra columna (`priceInfo`).

In [112]:
to_delete = [
	"Unnamed: 0",
	"n�mero de veces",
	"Nuda",
	"thumbnail",
	"externalReference",
	"url",
	"priceInfo",
	"dhasParking",
	"dsugTextTit",
	"dsugTexsubtit",
	"dParkingIncl",
	"dType",
	"distance"
]
df_raw.drop(columns=to_delete, inplace=True)

#### Normalización de columnas

Extraemos los datos de las columnas que tienen un `dict` como valor.

In [113]:
columns_to_normalize = [
	"labels",
	"highlight",
	"parkingSpace",
	"detailedType",
	"suggestedTexts"
]
df_raw[columns_to_normalize].sample(15)

,labels,highlight,parkingSpace,detailedType,suggestedTexts
13198,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",{'typology': 'flat'},"{'subtitle': 'Ciudad Universitaria, Madrid', '..."
6921,NaN,NaN,NaN,{'typology': 'flat'},"{'subtitle': 'Ibiza, Madrid', 'title': 'Piso'}"
29053,NaN,{'groupDescription': 'Top'},NaN,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Palacio, Madrid', 'title': 'Átic..."
709,NaN,{'groupDescription': 'Top'},NaN,{'typology': 'flat'},"{'subtitle': 'Recoletos, Madrid', 'title': 'Pi..."
9216,"[{'name': 'luxuryType', 'text': 'Lujo'}]",{'groupDescription': 'Top'},"{'hasParkingSpace': True, 'isParkingSpaceInclu...","{'typology': 'chalet', 'subTypology': 'terrace...","{'subtitle': 'El Viso, Madrid', 'title': 'Chal..."
24680,NaN,NaN,NaN,{'typology': 'flat'},"{'subtitle': 'Palos de Moguer, Madrid', 'title..."
16489,NaN,{'groupDescription': 'Top'},NaN,{'typology': 'flat'},"{'subtitle': 'Pacífico, Madrid', 'title': 'Pis..."
25923,NaN,{'groupDescription': 'Destacado'},NaN,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Delicias, Madrid', 'title': 'Áti..."
6437,NaN,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",{'typology': 'flat'},"{'subtitle': 'Almagro, Madrid', 'title': 'Piso..."
14774,NaN,NaN,NaN,{'typology': 'flat'},"{'subtitle': 'Valdeacederas, Madrid', 'title':..."


La columna `labels` contiene una lista. Debemos realizar un aplanamiento de la misma antes de realizar alguna otra modificación.

In [114]:
gt_one_element = df_raw["labels"].apply(
		lambda x: len(x) if isinstance(x, list) else 0) > 1
gt_one_element.value_counts()

labels
False    29922
Name: count, dtype: int64

In [115]:
df_raw.loc[gt_one_element, "labels"].values

array([], dtype=object)

Como vemos, existen algunos pocos registros donde la columna `labels` tiene más de un elemento en ella. Creamos una función que nos ayude a resolver el cometido.

In [116]:
def flaten(lists):
	if isinstance(lists, list) and len(value) > 1:
		return {
			f"{k}_{i}": v 
			for i, element in enumerate(lists)
			for k, v in element.items()
		}
	else:
		return

In [117]:
df_raw["labels"] = df_raw["labels"].apply(flaten)
df_raw["labels"]

0        None
1        None
2        None
3        None
4        None
         ... 
29917    None
29918    None
29919    None
29920    None
29921    None
Name: labels, Length: 29922, dtype: object

Y procedemos con la normalización de todas las columnas...

In [118]:
normalized_dataframes = []
for column in columns_to_normalize:
	print("Normalizando columna:",column)
	normalized = pd.json_normalize(df_raw[column])
	df_raw = df_raw.join(normalized,
						 how="left",
						 lsuffix="_left",
						 rsuffix="_right")

Normalizando columna: labels
Normalizando columna: highlight
Normalizando columna: parkingSpace
Normalizando columna: detailedType
Normalizando columna: suggestedTexts


In [119]:
columns_review = [
	"labels",
	"name_0",
	"text_0",
	"name_1",
	"text_1",
	"highlight",
	"groupDescription",
	"parkingSpace",
	"hasParkingSpace",
	"isParkingSpaceIncludedInPrice",
	"parkingSpacePrice",
	"detailedType",
	"typology",
	"subTypology",
	"suggestedTexts",
	"subtitle",
	"title"
]
df_raw[columns_review].sample(10)

KeyError: "['name_0', 'text_0', 'name_1', 'text_1', 'groupDescription', 'hasParkingSpace', 'isParkingSpaceIncludedInPrice', 'parkingSpacePrice', 'typology', 'subTypology', 'subtitle', 'title'] not in index"

In [122]:
df_raw.to_csv(f"{base_path}/df_raw_v1.csv", sep=",")

In [125]:
df_raw.columns

Index(['Unnamed: 0.1', 'propertyCode', 'numPhotos', 'floor', 'price',
       'propertyType', 'operation', 'size', 'exterior', 'rooms', 'bathrooms',
       'address', 'province', 'municipality', 'district', 'country',
       'neighborhood', 'latitude', 'longitude', 'showAddress', 'description',
       'hasVideo', 'status', 'newDevelopment', 'hasLift', 'parkingSpace',
       'priceByArea', 'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour',
       'has360', 'hasStaging', 'highlight', 'topNewDevelopment',
       'superTopHighlight', 'labels', 'newDevelopmentFinished',
       'newDevelopment.1', 'SubdType', 'ex', 'topPlus'],
      dtype='object')

In [141]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29922 entries, 0 to 29921
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0.1            29922 non-null  int64  
 1   propertyCode            29917 non-null  float64
 2   numPhotos               9571 non-null   float64
 3   floor                   27141 non-null  object 
 4   price                   29917 non-null  float64
 5   propertyType            29917 non-null  object 
 6   operation               29917 non-null  object 
 7   size                    29917 non-null  float64
 8   exterior                28873 non-null  object 
 9   rooms                   29917 non-null  float64
 10  bathrooms               29917 non-null  float64
 11  address                 29917 non-null  object 
 12  province                29917 non-null  object 
 13  municipality            29917 non-null  object 
 14  district                29917 non-null

## Tabla de variables


| Columna/Variable        | Descripción                                      | Importancia inicial | Notas                           |
|-------------------------|--------------------------------------------------|---------------------|---------------------------------|
| Unnamed: 0.1            | Índice generado automáticamente                  | -               | Puede eliminarse                |
| propertyCode            | Código único del inmueble                        | 2                | Identificador principal         |
| numPhotos               | Número de fotos del anuncio                      | 2              | Útil para calidad del anuncio   |
| floor                   | Planta del inmueble                              | 1             | Puede tener valores no estándar |
| price                   | Precio total                                     | 0               | Variable objetivo frecuente     |
| propertyType            | Tipo de propiedad                                | 1             | Categórica                      |
| operation               | Tipo de operación (venta/alquiler)               | 1               | Categórica                      |
| size                    | Metros cuadrados                                 | 1               | Numérica                        |
| exterior                | Si es exterior o interior                        | 2              | Binaria                         |
| rooms                   | Número de habitaciones                           | 1               | Numérica                        |
| bathrooms               | Número de baños                                  | 1                | Numérica                        |
| address                 | Dirección completa                               | 1              | Puede estar oculta              |
| province                | Provincia                                        | 1              | Categórica                      |
| municipality            | Municipio                                        | 1             | Categórica                      |
| district                | Distrito                                         | 1              | Categórica                      |
| country                 | País                                             | 1                | Normalmente constante           |
| neighborhood            | Barrio                                           | 1              | Categórica                      |
| latitude                | Coordenada latitud                               | 3              | Geolocalización                 |
| longitude               | Coordenada longitud                              | 3               | Geolocalización                 |
| showAddress             | Indica si la dirección es visible                | -               | Binaria                         |
| description             | Descripción del anuncio                          | -               | Texto libre                     |
| hasVideo                | Si tiene video                                   | 3               | Binaria                         |
| status                  | Estado del inmueble                              | 2               | Categórica                      |
| newDevelopment          | Si es obra nueva                                 | 1               | Binaria                         |
| hasLift                 | Si tiene ascensor                                | 1              | Binaria                         |
| parkingSpace            | Información sobre parking                        | 1               | Puede ser dict o booleano       |
| priceByArea             | Precio por metro cuadrado                        | 0               | Derivada                        |
| detailedType            | Tipo detallado                                   | 1               | Categórica                      |
| suggestedTexts          | Textos sugeridos                                 | -               | Estructura compleja             |
| hasPlan                 | Si tiene plano                                   | 1                | Binaria                         |
| has3DTour               | Si tiene tour 3D                                 | 3               | Binaria                         |
| has360                  | Si tiene vista 360                               | 3               | Binaria                         |
| hasStaging              | Si tiene staging virtual                          | 3               | Binaria                         |
| highlight               | Destacado en la plataforma                       | 3             | Binaria                         |
| topNewDevelopment       | Destacado como obra nueva                        | 1               | Binaria                         |
| superTopHighlight       | Súper destacado                                  | 3              | Binaria                         |
| labels                  | Etiquetas del anuncio                            | -              | Lista o categórica              |
| newDevelopmentFinished  | Obra nueva finalizada                            | 2               | Binaria                         |
| newDevelopment.1        | Duplicado o variable relacionada                 | 2                | Revisar redundancia             |
| SubdType                | Subtipo del inmueble                             | 2               | Categórica                      |
| ex                      | Variable auxiliar                                | -               | Revisar significado             |
| topPlus                 | Destacado premium                                | 2              | Binaria                         |


## Análisis univariante

In [126]:
df_tipificacion = pd.DataFrame([df_raw.nunique(), df_raw.nunique()/len(df_raw) * 100, df_raw.dtypes]).T.rename(columns = {0: "Card",\
                                                                                                                                 1: "%_Card", 2: "Tipo"})
df_tipificacion

,Card,%_Card,Tipo
Unnamed: 0.1,29922,100.0,int64
propertyCode,4459,14.902079,float64
numPhotos,102,0.340886,float64
floor,25,0.083551,object
price,1073,3.58599,float64
propertyType,5,0.01671,object
operation,1,0.003342,object
size,416,1.390281,float64
exterior,2,0.006684,object
rooms,12,0.040104,float64


In [128]:
df_tipificacion["Clasificada_como"] = "Categorica" 
df_tipificacion.loc[df_tipificacion.Card == 2, "Clasificada_como"] = "Binaria"
df_tipificacion.loc[df_tipificacion["Card"] > 10, "Clasificada_como"] ="Numerica Discreta"
df_tipificacion.loc[df_tipificacion["%_Card"] > 30, "Clasificada_como"] = "Numerica Continua"
df_tipificacion

,Card,%_Card,Tipo,Clasificada_como
Unnamed: 0.1,29922,100.0,int64,Numerica Continua
propertyCode,4459,14.902079,float64,Numerica Discreta
numPhotos,102,0.340886,float64,Numerica Discreta
floor,25,0.083551,object,Numerica Discreta
price,1073,3.58599,float64,Numerica Discreta
propertyType,5,0.01671,object,Categorica
operation,1,0.003342,object,Categorica
size,416,1.390281,float64,Numerica Discreta
exterior,2,0.006684,object,Binaria
rooms,12,0.040104,float64,Numerica Discreta


In [ ]:
df_raw["ex"].value_counts() # Eliminar

ex
1068.0    2
650.0     2
844.0     2
653.0     2
49.0      2
         ..
1036.0    2
340.0     2
343.0     2
1197.0    1
1196.0    1
Name: count, Length: 1198, dtype: int64

In [133]:
df_raw["detailedType"].value_counts()

detailedType
{'typology': 'flat'}                                          25077
{'typology': 'flat', 'subTypology': 'penthouse'}               2245
{'typology': 'flat', 'subTypology': 'duplex'}                   959
{'typology': 'chalet', 'subTypology': 'independantHouse'}       538
{'typology': 'chalet', 'subTypology': 'terracedHouse'}          444
{'typology': 'flat', 'subTypology': 'studio'}                   372
{'typology': 'chalet', 'subTypology': 'semidetachedHouse'}      225
{'typology': 'chalet'}                                           57
Name: count, dtype: int64

In [136]:
df_raw["labels"].value_counts()

Series([], Name: count, dtype: int64)

In [139]:
df_raw["newDevelopment.1"].value_counts()

newDevelopment.1
False    19657
True       689
Name: count, dtype: int64

### Columnas Agustín/ detailedType- topPlus 


### Columnas Roberto/ PropertyCode-Province

### Columnas Sofía/ municipality-priceByArea